In [11]:
import numpy as np
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import torchvision
from torchvision import transforms as T

import tqdm

import warnings
warnings.filterwarnings('ignore')

In [12]:
path_data = './dataset/resized'
device = 'cuda'
batch_size = 16

In [15]:
class ImagesDS(D.Dataset):
    def __init__(self, csv_file, img_dir, mode='train', site=1, channels=[1,2,3,4,5,6]):
        """
        Args:
            csv_file (string): Path to the csv file with image metadata
            img_dir (string): Directory with all the images
            mode (string): 'train' or 'test'
            site (int): Site number for the image
            channels (list): List of channels to use
        """
        df = pd.read_csv(csv_file)
        self.records = df.to_records(index=False)
        self.channels = channels
        self.site = site
        self.mode = mode
        self.img_dir = img_dir
        self.len = df.shape[0]

    @staticmethod
    def _load_img_as_tensor(file_name):
        """Load image and convert to tensor"""
        with Image.open(file_name) as img:
            return T.ToTensor()(img)

    def _get_img_path(self, index, channel):
        """Get path for a specific image"""
        experiment = self.records[index].experiment
        well = self.records[index].well
        plate = self.records[index].plate
        return '/'.join([
            self.img_dir,
            self.mode,
            experiment,
            f'Plate{plate}',
            f'{well}_s{self.site}_w{channel}.png'
        ])

    def __getitem__(self, index):
        """Get a sample from the dataset"""
        paths = [self._get_img_path(index, ch) for ch in self.channels]

        img = torch.cat([self._load_img_as_tensor(img_path) for img_path in paths])

        if self.mode == 'train':
            return img, self.records[index].sirna
        else:
            return img, self.records[index].id_code

    def __len__(self):
        """Total number of samples in the dataset"""
        return self.len

In [16]:
ds = ImagesDS(path_data+'/train.csv', path_data)
ds_test = ImagesDS(path_data+'/test.csv', path_data, mode='test')


FileNotFoundError: [Errno 2] No such file or directory: './dataset/resized/train.csv'